In [82]:
import pandas as pd
import os
import numpy as np

In [83]:
data_dir = '/home/victorialin/Documents/2022-2023/causal_text/data/emobank/'
outcome = 'D_writer'
outcome_reader = 'D_reader'
outcome_name = 'dominance'

In [84]:
emobank = pd.read_csv(os.path.join(data_dir, 'emobank.csv'))
df_writer = pd.read_csv(os.path.join(data_dir, 'writer.csv'))
df_reader = pd.read_csv(os.path.join(data_dir, 'reader.csv'))

In [85]:
emobank.columns = ['id', 'split', 'V_combined', 'A_combined', 'D_combined', 'text']
emobank.head()

,id,split,V_combined,A_combined,D_combined,text
0,110CYL068_1036_1079,train,3.00,3.00,3.20,"Remember what she said in my last letter? """
1,110CYL068_1079_1110,test,2.80,3.10,2.80,If I wasn't working here.
2,110CYL068_1127_1130,train,3.00,3.00,3.00,".."""
3,110CYL068_1137_1188,train,3.44,3.00,3.22,Goodwill helps people get off of public assist...
4,110CYL068_1189_1328,train,3.55,3.27,3.46,Sherry learned through our Future Works class ...


In [86]:
df_writer.drop(['stdV', 'stdA', 'stdD', 'N'], axis=1, inplace=True)
df_writer.columns = ['id', 'V_writer', 'A_writer', 'D_writer']
df_writer.head()

,id,V_writer,A_writer,D_writer
0,110CYL068_1036_1079,3.00,2.8,3.4
1,110CYL068_1079_1110,3.00,3.2,3.0
2,110CYL068_1127_1130,3.00,3.0,3.0
3,110CYL068_1137_1188,3.25,3.0,3.0
4,110CYL068_1189_1328,3.40,3.4,3.2


In [87]:
df_reader.drop(['stdV', 'stdA', 'stdD', 'N'], axis=1, inplace=True)
df_reader.columns = ['id', 'V_reader', 'A_reader', 'D_reader']
df_reader.head()

,id,V_reader,A_reader,D_reader
0,110CYL068_1036_1079,3.0,3.20,3.00
1,110CYL068_1079_1110,2.6,3.00,2.60
2,110CYL068_1110_1127,2.0,2.33,2.33
3,110CYL068_1127_1130,3.0,3.00,3.00
4,110CYL068_1137_1188,3.6,3.00,3.40


In [88]:
df_combined = emobank.merge(df_writer.merge(df_reader, on='id'), on='id')
df_combined.head()

,id,split,V_combined,A_combined,D_combined,text,V_writer,A_writer,D_writer,V_reader,A_reader,D_reader
0,110CYL068_1036_1079,train,3.00,3.00,3.20,"Remember what she said in my last letter? """,3.00,2.8,3.4,3.00,3.20,3.00
1,110CYL068_1079_1110,test,2.80,3.10,2.80,If I wasn't working here.,3.00,3.2,3.0,2.60,3.00,2.60
2,110CYL068_1127_1130,train,3.00,3.00,3.00,"..""",3.00,3.0,3.0,3.00,3.00,3.00
3,110CYL068_1137_1188,train,3.44,3.00,3.22,Goodwill helps people get off of public assist...,3.25,3.0,3.0,3.60,3.00,3.40
4,110CYL068_1189_1328,train,3.55,3.27,3.46,Sherry learned through our Future Works class ...,3.40,3.4,3.2,3.67,3.17,3.67


In [89]:
writer_median = df_combined[outcome].median()

In [99]:
writer_median

3.0

In [90]:
df_combined = df_combined[df_combined[outcome] != writer_median]

In [91]:
df_combined['P(V>3)'] = 0.2
df_combined['P(V<3)'] = 0.2
df_combined.loc[df_combined[outcome] > writer_median, 'P(V>3)'] = 0.8
df_combined.loc[df_combined[outcome] < writer_median, 'P(V<3)'] = 0.8

In [92]:
df_combined.head()

,id,split,V_combined,A_combined,D_combined,text,V_writer,A_writer,D_writer,V_reader,A_reader,D_reader,P(V>3),P(V<3)
0,110CYL068_1036_1079,train,3.00,3.00,3.20,"Remember what she said in my last letter? """,3.00,2.8,3.40,3.00,3.20,3.00,0.8,0.2
4,110CYL068_1189_1328,train,3.55,3.27,3.46,Sherry learned through our Future Works class ...,3.40,3.4,3.20,3.67,3.17,3.67,0.8,0.2
5,110CYL068_1475_1558,train,3.60,3.30,3.80,Coming to Goodwill was the first step toward m...,3.60,3.6,3.60,3.60,3.00,4.00,0.8,0.2
6,110CYL068_1559_1601,train,3.00,3.00,3.10,"I am now... totally off of welfare.""",3.40,3.0,3.20,2.60,3.00,3.00,0.8,0.2
9,110CYL068_1659_1810,train,3.25,3.00,3.25,Cornell found a number of employment options t...,3.33,3.0,3.67,3.20,3.00,3.00,0.8,0.2


In [93]:
df_highwriterintent = df_combined.sample(n=5000, replace=True, weights='P(V>3)', random_state=230425)
df_lowwriterintent = df_combined.sample(n=5000, replace=True, weights='P(V<3)', random_state=102938)

In [94]:
df_highwriterintent[outcome].mean()

3.263924000000095

In [100]:
df_lowwriterintent[outcome].mean()

2.939585999999962

In [95]:
(df_lowwriterintent[outcome] > 3).mean()

0.3748

In [96]:
df_highwriterintent[outcome_reader].mean()

3.085574000000013

In [97]:
df_lowwriterintent[outcome_reader].mean()

3.0079320000000034

In [98]:
df_highwriterintent.to_csv(os.path.join(data_dir, 'highwriterintent_{}.csv'.format(outcome_name)), index=False)
df_lowwriterintent.to_csv(os.path.join(data_dir, 'lowwriterintent_{}.csv'.format(outcome_name)), index=False)